In [20]:
%pip install tensorflow 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn import metrics 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
data = pd.read_csv("banks.csv",sep="\t") 
data.head

<bound method NDFrame.head of          idx     Score  \
0          0  Positive   
1          1  Negative   
2          2  Positive   
3          3  Negative   
4          4  Negative   
...      ...       ...   
13994  13994  Positive   
13995  13995  Positive   
13996  13996  Positive   
13997  13997  Positive   
13998  13998  Negative   

      Text,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,  
0      В Альфа-Банке работает замечательная девушка -...                                                                                                                     

In [3]:

X_train, X_test, y_train, y_test = train_test_split(data['Text,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,'], data['Score'], test_size=0.2, random_state=42)

 Почему наивный байесовский  классификатор ?  
 - эфективность в работе с текстовыми данными из за большого колличества признаков  
 - скорость обучения 

In [8]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [9]:
y_pred = model.predict(X_test)

In [10]:
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9225


92% поидее неплохой результат 


In [12]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [13]:
max_words = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test) 
 
max_len = 100 
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [14]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=32, input_length=max_len))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 
model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))


Epoch 1/5


350/350 [==============================] - 2s 5ms/step - loss: 0.3263 - accuracy: 0.8534 - val_loss: 0.2261 - val_accuracy: 0.9093
Epoch 2/5
350/350 [==============================] - 2s 5ms/step - loss: 0.1305 - accuracy: 0.9547 - val_loss: 0.2547 - val_accuracy: 0.9007
Epoch 3/5
350/350 [==============================] - 1s 4ms/step - loss: 0.0384 - accuracy: 0.9900 - val_loss: 0.3423 - val_accuracy: 0.9018
Epoch 4/5
350/350 [==============================] - 2s 4ms/step - loss: 0.0077 - accuracy: 0.9990 - val_loss: 0.3928 - val_accuracy: 0.8954
Epoch 5/5
350/350 [==============================] - 1s 4ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 0.4267 - val_accuracy: 0.8975


In [16]:
y_pred_test = model.predict(X_test_pad)
y_pred = (y_pred_test > 0.5).astype(int)

88/88 [==============================] - 0s 2ms/step


In [17]:
print("Precision:", metrics.precision_score(y_test, y_pred))

Precision: 0.9005763688760807


Тут слой вложения и слой выравнивания , потом они подаются на полносвязные слои 